In [2]:
# %load scripts/document_status.py
import sys
import os
import re
import pandas as pd
import redis
from pathlib import Path
sys.path.append('/home/jeremy/Library')

from storage.cherrytree import CherryTree

r = redis.Redis(decode_responses=True)
project_index_key = 'projects:filepath.document:index'

def get_document_metadata(node):
    filepath = Path(node.filepath).resolve()
    document_key = r.hget(project_index_key, str(filepath))
    if not document_key:
        return None
    document_data_key = r.hget(document_key, 'metadata')
    if not document_data_key:
        return None
    data = {k:v for k,v in r.hgetall(document_data_key).items()}
    if not data:
        return None
    return dict(node=node, data=data)



In [29]:
ct = CherryTree('synopsis.ctd')
df = pd.DataFrame(filter(None, [get_document_metadata(n) for n in ct.nodes('Stories') if n.filepath]))
df['name'] = df.node.apply(lambda x: x.name)
df['status'] = df.data.apply(lambda x: x.get('status'))

In [20]:

df.head()

,node,data,name,status
0,<storage.cherrytree.Node object at 0x7f1fc56cd...,"{'title': 'Lunch with Julia', 'identifier': '7...",Lunch With Julia,Final
1,<storage.cherrytree.Node object at 0x7f1fc56cd...,"{'date': 'October 1988', 'project': 'omaf', 't...",Julia Rides a Harley,Final
2,<storage.cherrytree.Node object at 0x7f1fc56cd...,"{'status': 'Final', 'date': 'October 1988', 'l...",Bule Kaya,Final
3,<storage.cherrytree.Node object at 0x7f1fc56d4...,"{'status': 'Uncertain', 'date': 'October 1988'...",Julia at the Tanamur,Uncertain
4,<storage.cherrytree.Node object at 0x7f1fc56d4...,"{'location': 'Borobudur', 'status': 'Rewrite',...",Date With Eric,Rewrite


In [31]:
df[~df.status.str.contains('Final|Hold')][['name', 'status']]


,name,status
4,Date With Eric,Rewrite
6,Julia Finds Photo,Expand
12,Syd Warns Cameron,Rewrite and place
15,Syd Reveals History,Expand and place
16,Useful Idiot,Rewrite and place
17,Unequal Friendship,Expand
18,Sabam Furious,Expand
19,Syd Dies,Expand
20,Victor Learns Truth,Expand
21,Sabam Fires Cameron,Expand


In [ ]:
loadpy scripts/match_stories_and_files